
Source: https://networkrepository.com/fb-pages-politician.php

In [1]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
from graphframes import GraphFrame
import os

spark = SparkSession.builder.appName("graphs")\
        .config('spark.jars.packages', 'io.graphframes:graphframes-spark4_2.13:0.10.0')\
        .master("local[*]")\
        .getOrCreate()

:: loading settings :: url = jar:file:/home/pol/dev/zajecia/pyspark-graphs/.venv/lib/python3.12/site-packages/pyspark/jars/ivy-2.5.3.jar!/org/apache/ivy/core/settings/ivysettings.xml
Ivy Default Cache set to: /home/pol/.ivy2.5.2/cache
The jars for the packages stored in: /home/pol/.ivy2.5.2/jars
io.graphframes#graphframes-spark4_2.13 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-ef9f1722-4b77-4061-a457-1808efcd555f;1.0
	confs: [default]
	found io.graphframes#graphframes-spark4_2.13;0.10.0 in central
	found io.graphframes#graphframes-graphx-spark4_2.13;0.10.0 in central
:: resolution report :: resolve 67ms :: artifacts dl 3ms
	:: modules in use:
	io.graphframes#graphframes-graphx-spark4_2.13;0.10.0 from central in [default]
	io.graphframes#graphframes-spark4_2.13;0.10.0 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       c

In [2]:
def toHtml(self):
    """Returns a dataframe - supports spark lines"""
    import cgi
    from itertools import islice
    if not self._support_repr_html:
        self._support_repr_html = True
    max_num_rows = max(self.sql_ctx._conf.replEagerEvalMaxNumRows(), 0)
    sock_info = self._jdf.getRowsToPython(
    max_num_rows, self.sql_ctx._conf.replEagerEvalTruncate())
    rows = islice(self.toLocalIterator(), 0, max_num_rows)
 
    html = "</pre>\n" #generate table header html += " \n" % " ".join(map(lambda x: cgi.escape(x), df.columns)) #generate table rows for row in rows: html += " \n" % "".join(map(lambda x: str(x), row)) 
    html += '<table border="1"><tbody><tr><th>%s</th></tr><tr><td>%s</td></tr></tbody></table><pre>\n'
    html += "only showing upto %d %s\n" % (max_num_rows, "row" if max_num_rows == 1 else "rows")
    return html

from pyspark.sql.dataframe import DataFrame
DataFrame._repr_html_ = toHtml
spark.conf.set("spark.sql.repl.eagerEval.enabled", True)

In [3]:
# File location and type
file_location = "data/fb_pages_politician.edges"
file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "false"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df_edges = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)
df_edges = df_edges.withColumnRenamed('_c0', 'src')
df_edges = df_edges.withColumnRenamed('_c1', 'dst')
df_edges.show()

+---+----+
|src| dst|
+---+----+
|  0|1972|
|  0|5111|
|  0| 138|
|  0|3053|
|  0|1473|
|  0|4978|
|  0|4523|
|  0|2334|
|  0|1038|
|  0|3923|
|  0|3874|
|  0| 766|
|  0|  98|
|  0|2676|
|  0|1547|
|  0|3622|
|  0|5292|
|  0|5455|
|  0|2755|
|  0| 568|
+---+----+
only showing top 20 rows


In [4]:
# File location and type
file_location = "data/fb_pages_politician.nodes"
file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df_nodes = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

df_nodes = df_nodes.withColumnRenamed('id', 'fb_id')
df_nodes = df_nodes.withColumnRenamed('new_id', 'id')
	
display(df_nodes)

fb_id,name,id
127397457601670,陳根德,1661
169014523134260,林淑芬,52
295363753921281,Chris White,3477
613631835423903,Stewart Hosie MP,3193
284882801581896,Frank Schäffler,4425
712493248818711,Jean-François Jalkh,2130
105367536223828,Kennedy Stewart,1716
561507140593570,Hans Andersen,4812
1454302588159101,Scott Emerson,3185
585862021507535,Vittorio Ferraresi,4903


In [5]:
from graphframes import *
g = GraphFrame(df_nodes, df_edges)

In [6]:
display(g.inDegrees)

id,inDegree
1088,6
4519,13
5518,8
2142,1
4935,4
2366,16
5156,13
1591,2
496,2
2122,3


In [7]:
display(g.outDegrees)

id,outDegree
148,2
463,3
471,12
496,18
833,2
1088,23
1238,12
1342,8
1580,8
1645,2


In [8]:
print("triangle count")
from graphframes.classic.graphframe import StorageLevel
result = g.triangleCount(storage_level=StorageLevel.MEMORY_ONLY)
display(result.sort('count', ascending=False))

triangle count


26/01/12 17:45:19 WARN BlockManager: Block rdd_71_0 already exists on this machine; not re-adding it
26/01/12 17:45:21 WARN AggregateMessages: Returned DataFrame is persistent and materialized!
26/01/12 17:45:25 WARN TriangleCount$: Returned DataFrame is persistent and materialized!


fb_id,name,id,src,src_triplets,dst,dst_triplets,count
1522202731410785,Katarina Barley,5416,5416,807,5416,6547,3677
480197465350951,Arno Klare MdB,1595,1595,5333,1595,1819,3576
499836743410538,Katja Mast,4602,4602,1848,4602,5234,3541
1475998419282812,Heike Baehrens,4972,4972,1193,4972,5375,3284
664443296948057,Christian Petry,1414,1414,5032,1414,1354,3193
439265522793389,Michelle Müntefering,2770,2770,2865,2770,2513,2689
104493292916735,Sigmar Gabriel,726,726,5026,726,316,2671
334258583436365,Klaus Mindrup,5726,5726,299,5726,4941,2620
601163706652420,Niels Annen,4032,4032,2002,4032,3056,2529
464998863536680,Bill Shorten MP,98,98,4835,98,109,2472


In [9]:
print("page rank")
result = g.pageRank(resetProbability=0.15, tol=0.1)
#display(result)
result.vertices.sort("pagerank", ascending=[False]).select("id", "name", "pagerank").show()

page rank


26/01/12 17:45:26 WARN ShippableVertexPartitionOps: Joining two VertexPartitions with different indexes is slow.


+----+--------------------+------------------+
|  id|                name|          pagerank|
+----+--------------------+------------------+
|5800|        Barack Obama|  66.0969264689174|
|5873|         Frank Junge| 45.71996784757828|
|5871|    Johannes Schraps| 39.83364047860558|
|5882|        Peter Tauber|34.337987353253915|
|5855| Anthony Albanese MP| 27.22534049334262|
|5888| Gary Anandasangaree|26.913589304797213|
|5899|Dr. Nicole Hoffme...|25.338413029723462|
|4874|    Joachim Herrmann|19.983483977948886|
|5867|        Dr. Hedy Fry| 19.72310976319604|
|5879|       Majid Jowhari|19.572472228089698|
|2981|       Tulsi Gabbard|18.300684944277666|
|5900|    Pierre Poilievre|18.056612320079477|
|5726|       Klaus Mindrup|17.231659243611873|
|5416|     Katarina Barley|16.010828422080426|
|5868|Christina Jantz-H...|15.905154641013574|
|5898|     Kurt Wiegel MdL|15.105129519946301|
|5797|       Hubertus Heil| 14.11076843869615|
|5885|     Deborah O'Neill| 13.74942377741333|
|5889|  Feder

26/01/12 17:45:27 WARN PageRank: Returned DataFrame is persistent and materialized!


In [10]:
print("connected components")
result = g.connectedComponents(algorithm="graphx")
#display(result)
display(result.sort('component', ascending=False))
# How many clusters / connected components in total for this dataset
#cluster_num = result.select("component").distinct().count()
#print("clusters amount: ", cluster_num)
#print()


connected components


fb_id,name,id,component
608246322520640,Rogério Peninha M...,4551,0
1722841861328522,Nick Serianni - L...,667,0
308920973786,Joanna Mucha,5618,0
140656636910,José María Figuer...,5354,0
207423949711700,Kathalijne Buitenweg,1894,0
621011257931252,Fritz Güntzler MdB,2493,0
134629609925206,Erik Ullenhag,4926,0
23182996671,Dora Bakoyannis -...,5384,0
1695475020740767,Fatma Øktem,3877,0
232372840163384,Charles Mok 莫乃光,4839,0


In [11]:
import networkx as nx
graph_pandas = g.edges.toPandas()
print(graph_pandas)
nxg = nx.from_pandas_edgelist(graph_pandas, 'src', 'dst')
#nx.draw(nxg, with_labels=False)

        src   dst
0         0  1972
1         0  5111
2         0   138
3         0  3053
4         0  1473
...     ...   ...
41724  5871  5873
41725  5879  5888
41726  5882  5899
41727  5882  5898
41728  5886  2981

[41729 rows x 2 columns]


In [ ]:
from pyvis.network import Network
from IPython.display import HTML
import networkx as nx

graph_pandas = g.edges.toPandas()
G = nx.MultiDiGraph()
nxg = nx.from_pandas_edgelist(graph_pandas, 'src', 'dst')
#G.add_edges_from(list((, n) for n, in G.graph_pandas))
G.add_edges_from([(r.src, r.dst) for r in g.edges.collect()])
nx.draw(G, with_labels=True)
nx.draw(G, with_labels=True, connectionstyle='arc3, rad = 0.1')

nt = Network(notebook=True, directed=True)
nt.from_nx(G)
nt.prep_notebook()
HTML(nt.generate_html())
